Projeto Final Pós Graduação Big Data - Senac Rio - 2020.

Processo de importação das Bibliotecas utilizadas no durante todo o projeto.

In [1]:
#Importação das bibliotecas utilizadas para manipulação dos dados das paginas Web.

from bs4 import BeautifulSoup
import urllib.request
import shutil
import os
from datetime import datetime
import pandas as pd
import pyspark
import zipfile
import sqlalchemy
import pymysql
import json
import numpy as np
import matplotlib.pyplot as plt
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import * 

Indicação do diretório para download dos arquivos Inep-Enem. 

In [ ]:
#Indicação do diretório desejado para download dos arquivos. 
#No exemplo, estamos salvando no diretório: C:\Users\Ricardo\Downloads

diretorio = input('Digite o diretório desejado para salvar os arquivos: ')
os.chdir(r'{}'.format(diretorio))
os.getcwd( )

In [ ]:
#Apos indicar o diretorio desejado, como boa pratica, sugerimos a criação da pasta onde será salvos os arquivos.
#No exemplo, estamos salvando na pasta: MicrodadosEnem.'''

url = 'http://inep.gov.br/microdados'

pasta = input('Após indicação do diretório {}, informe o nome da pasta desejada para salvar os arquivos baixados da pagina {}: '.format(diretorio, url))

try:
    os.mkdir(pasta)
    print('Pasta {} criada com sucesso!'.format(pasta))
except FileExistsError as e:
    print(f'Pasta {pasta} já existe')

In [ ]:
#Indicação do diretório onde será salvo os arquivos baixados da url.

os.chdir(r'{}\{}'.format(diretorio, pasta))
print('Os arquivos serão salvos no seguinte diretorio:', format(os.getcwd()))

Processo de Web Scraping no site do Inep, buscando os microdados do Enem.

In [ ]:
#Criação da variável que receberá todos os dados correspondente a varíavel "url".

page = urllib.request.urlopen(url)

In [ ]:
#Criação da variável "soup", do pacote BeatifulSoup, que apresenta a estutura HTML da pagina que será trabalhada.

soup = BeautifulSoup(page,'html.parser')
print(soup)

In [ ]:
#No loop "for" abaixo, o realizaremos o filtro das tags desejadas.

for link in soup.findAll('a'):
    theLink = link.get('href')
    name = link.string
    
    print(theLink)
    print(name)

In [ ]:
#No processo abaixo, realizaremos o filtro das linhas, cujo o o tipo do arquivo desejado é *.zip.

nomeArquivo = []
typeFile = '.zip'
i = 0
for link in soup.findAll('a'):
    theLink = str(link.get('href')).lower()
    name = link.string
    if 'enem'in theLink:
        if theLink[-4:] == typeFile:
            print(theLink)
            print(type(theLink))
            print(name)
            print(type(name))
            nomeArquivo.append(theLink[39:-4].replace('/','_'))
            print(type(nomeArquivo))

In [ ]:
#Processo de download dos arquivos desejados e salvos na na pasta indicada.

i = 0
for link in soup.findAll('a'):
    theLink = str(link.get('href')).lower()
    name = link.string
    if 'enem'in theLink:
        if theLink[-4:] == typeFile:
            if name is None:
                title = 'SemTítulo' + typeFile
            else:
                title = nomeArquivo[i] + typeFile
                i +=1
            try:
                with open(os.getcwd() + '/' + title,'r') as fileExists:
                    print('Arquivo ' + title + ' existente no diretório '+ os.getcwd() + ' - ' + str(datetime.now()))
                    fileExists.close()
            except FileNotFoundError:
                print('Download Iniciado - {}: '.format(datetime.now()) + title)
                docFile = open(os.getcwd() + '/' + title,'wb')
                docFile.write(urllib.request.urlopen(theLink).read())
                docFile.close()
                print('Download Concluído em {}'.format(datetime.now()))
print('Não há mais arquivos para download')

In [ ]:
#Apresentação do tamanho dos arquivos baixados.

print('Apresentação Arquivos Enem baixados no díretório: ' + os.getcwd())
contador = 0
tamanhoTotal = 0
tipo = '.zip'

def formatarTamanho(tamanho):
    base = 1024
    kilo = base
    mega = base ** 2
    giga = base ** 3
    tera = base ** 4
    
    if tamanho < kilo:
        tamanho = base
        texto = 'B'
    elif tamanho < mega:
        tamanho /= kilo
        texto = 'K'
    elif tamanho < giga:
        tamanho /= mega
        texto = 'M'
    else:
        tamanho < tera
        tamanho /= giga
        texto = 'G'
    tamanho = round(tamanho, 2)
    return f'{tamanho} {texto}'

for raiz, diretorios, arquivos in os.walk(os.getcwd()):
    for arquivo in arquivos:
        if tipo in arquivo:
            contador +=1 
            caminho = os.path.join(raiz, arquivo)
            nome, ext = os.path.splitext(arquivo)
            tamanho = os.path.getsize(caminho)
            tamanhoTotal += tamanho
                                
            if tamanho > 0:
                print()
                print('Arquivo encontrado: ', arquivo)
                print('Caminho: ', caminho)
                print('Nome: ', nome)
                print('Extensão: ', ext)
                print('Tamanho: ', formatarTamanho(tamanho))

print()
print(f'{contador} arquivo(s) encontrado(s).' + 'Tamanho Total: ' + formatarTamanho(tamanhoTotal))

In [2]:
# Instanciar Spark

sc = SparkContext()
sqlContext = SQLContext(sc)

In [3]:
# No Processo abaixo, importaremos para o dataframe, apenas para conhecer os dados e verificar o separador do microdado.

print(f'Criação Dataframes Iniciado - {datetime.now()}.')
print('')

print('')
print(f'Criação Dataframe do Enem 2015 iniciada - {datetime.now()}.')
df_enem2015_completo = sqlContext.read.format('com.databricks.spark.csv').options(header='true', sep=",", inferschema='true', encoding = 'ISO-8859-1', ).load(r'C:\Users\Ricardo\Downloads\MicrodadosEnem\MICRODADOS_ENEM_2015.csv')
print(f'Dataframe df_enem2015_completo criado com sucesso - {datetime.now()}.')
print('')

print(f'Criação Dataframe do Enem 2016 iniciada- {datetime.now()}.')
df_enem2016_completo = sqlContext.read.format('com.databricks.spark.csv').options(header='true', sep=";", inferschema='true', encoding = 'ISO-8859-1').load(r'C:\Users\Ricardo\Downloads\MicrodadosEnem\microdados_enem_2016.csv')
print(f'Dataframe df_enem2016_completo criado com sucesso - {datetime.now()}.')
print('')

print(f'Criação Dataframe do Enem 2017 iniciada - {datetime.now()}.')
df_enem2017_completo = sqlContext.read.format('com.databricks.spark.csv').options(header='true', sep=";", inferschema='true', encoding = 'ISO-8859-1').load(r'C:\Users\Ricardo\Downloads\MicrodadosEnem\MICRODADOS_ENEM_2017.csv')
print(f'Dataframe df_enem2017_completo criado com sucesso - {datetime.now()}.')
print('')

print(f'Criação Dataframe do Enem 2018 iniciada - {datetime.now()}.')
df_enem2018_completo = sqlContext.read.format('com.databricks.spark.csv').options(header='true', sep=";", inferschema='true', encoding = 'ISO-8859-1').load(r'C:\Users\Ricardo\Downloads\MicrodadosEnem\MICRODADOS_ENEM_2018.csv')
print(f'Dataframe df_enem2018_completo criado com sucesso - {datetime.now()}.')
print('')

print(f'Criação Dataframe do Enem 2019 iniciada - {datetime.now()}.')
df_enem2019_completo = sqlContext.read.format('com.databricks.spark.csv').options(header='true', sep=";", inferschema='true', encoding = 'ISO-8859-1').load(r'C:\Users\Ricardo\Downloads\MicrodadosEnem\MICRODADOS_ENEM_2019.csv')
print(f'Dataframe df_enem2019_completo criado com sucesso - {datetime.now()}.')
print('')

print('')
print(f'Dataframes criados com sucesso - {datetime.now()}.')

Criação Dataframes Iniciado - 2020-08-20 15:05:01.461357.


Criação Dataframe do Enem 2015 iniciada - 2020-08-20 15:05:01.461357.
Dataframe df_enem2015_completo criado com sucesso - 2020-08-20 15:05:55.097559.

Criação Dataframe do Enem 2016 iniciada- 2020-08-20 15:05:55.097559.
Dataframe df_enem2016_completo criado com sucesso - 2020-08-20 15:06:54.440502.

Criação Dataframe do Enem 2017 iniciada - 2020-08-20 15:06:54.441499.
Dataframe df_enem2017_completo criado com sucesso - 2020-08-20 15:07:34.128135.

Criação Dataframe do Enem 2018 iniciada - 2020-08-20 15:07:34.128135.
Dataframe df_enem2018_completo criado com sucesso - 2020-08-20 15:08:06.571473.

Criação Dataframe do Enem 2019 iniciada - 2020-08-20 15:08:06.571473.
Dataframe df_enem2019_completo criado com sucesso - 2020-08-20 15:08:37.888182.


Dataframes criados com sucesso - 2020-08-20 15:08:37.889180.


In [4]:
# Apresentação dos registros de cada dataframe

print('Quantidade de Registros importadados para cada Dataframe')
print('')
print(f'Quantdade de Registros Dataframe Enem2015 - {datetime.now()}.')
print(df_enem2015_completo.count())
print('')
print(f'Quantdade de Registros Dataframe Enem2016 - {datetime.now()}.')
print(df_enem2016_completo.count())
print('')
print(f'Quantdade de Registros Dataframe Enem2017 - {datetime.now()}.')
print(df_enem2017_completo.count())
print('')
print(f'Quantdade de Registros Dataframe Enem2018 - {datetime.now()}.')
print(df_enem2018_completo.count())
print('')
print(f'Quantdade de Registros Dataframe Enem2019 - {datetime.now()}.')
print(df_enem2019_completo.count())


Quantidade de Registros importadados para cada Dataframe

Quantdade de Registros Dataframe Enem2015
7746427

Quantdade de Registros Dataframe Enem2016
8627367

Quantdade de Registros Dataframe Enem2017
6731341

Quantdade de Registros Dataframe Enem2018
5513747

Quantdade de Registros Dataframe Enem2019
5095270

Quantdade de Registros Dataframe Enem2019
5095270


In [5]:
# Apresentação de duas linhas do dataframe df_enem2015_completo

print(f'Apresentação de duas linhas do dataframe df_enem2015_completo - {datatime.now()}.')

df_enem2015_completo.show(n=2,truncate=True)

+------------+------+-----------------------+-----------------------+----------------+----------------+---------------------------+------------+---------+----------------+----------------+---------+---------+----------------------+------------------+---------------+--------+-------+----------------+-----------------------+-----------------------+----------------+----------------+---------------+---------------+---------+---------+---------------+-----------+--------------+-----------+---------+-----------------------+-----------------+---------------------+---------------------+------------------+-----------+-----------+-----------+--------+--------------+----------+------------------+------------+------------+--------------+--------------+----------+--------------+--------------+--------+---------+--------------+---------+-----------------+---------------------+------------------------+--------------+------------------+-------+-------------+-------------------+------------------+-----

In [6]:
# Apresentação de duas linhas do dataframe df_enem2015_completo

print(f'Apresentação de duas linhas do dataframe df_enem2015_completo, classificado como linhas - {datatime.now()}.')

df_enem2015_completo.head(2)

[Row(NU_INSCRICAO=150000000376, NU_ANO=2015, CO_MUNICIPIO_RESIDENCIA=4309209, NO_MUNICIPIO_RESIDENCIA='Gravataí', CO_UF_RESIDENCIA=43, SG_UF_RESIDENCIA='RS', IN_ESTUDA_CLASSE_HOSPITALAR=0, IN_TREINEIRO=0, CO_ESCOLA=None, CO_MUNICIPIO_ESC=None, NO_MUNICIPIO_ESC=None, CO_UF_ESC=None, SG_UF_ESC=None, TP_DEPENDENCIA_ADM_ESC=None, TP_LOCALIZACAO_ESC=None, TP_SIT_FUNC_ESC=None, NU_IDADE=42, TP_SEXO='M', TP_NACIONALIDADE=1, CO_MUNICIPIO_NASCIMENTO=4209003, NO_MUNICIPIO_NASCIMENTO='Joaçaba', CO_UF_NASCIMENTO=42, SG_UF_NASCIMENTO='SC', TP_ST_CONCLUSAO=1, TP_ANO_CONCLUIU=10, TP_ESCOLA=1, TP_ENSINO=None, TP_ESTADO_CIVIL=1, TP_COR_RACA=1, IN_BAIXA_VISAO=0, IN_CEGUEIRA=0, IN_SURDEZ=0, IN_DEFICIENCIA_AUDITIVA=0, IN_SURDO_CEGUEIRA=0, IN_DEFICIENCIA_FISICA=0, IN_DEFICIENCIA_MENTAL=0, IN_DEFICIT_ATENCAO=0, IN_DISLEXIA=0, IN_GESTANTE=0, IN_LACTANTE=0, IN_IDOSO=0, IN_DISCALCULIA=0, IN_AUTISMO=0, IN_VISAO_MONOCULAR=0, IN_SABATISTA=0, IN_OUTRA_DEF=0, IN_SEM_RECURSO=0, IN_NOME_SOCIAL=0, IN_BRAILLE=0, IN_AMP

In [7]:
# Apresentação da estutura do dataframe df_enem2015_completo

print(f'Apresentação da estutura do dataframe df_enem2015_completo - {datatime.now()}.')

df_enem2015_completo.printSchema()

root
 |-- NU_INSCRICAO: long (nullable = true)
 |-- NU_ANO: integer (nullable = true)
 |-- CO_MUNICIPIO_RESIDENCIA: integer (nullable = true)
 |-- NO_MUNICIPIO_RESIDENCIA: string (nullable = true)
 |-- CO_UF_RESIDENCIA: integer (nullable = true)
 |-- SG_UF_RESIDENCIA: string (nullable = true)
 |-- IN_ESTUDA_CLASSE_HOSPITALAR: integer (nullable = true)
 |-- IN_TREINEIRO: integer (nullable = true)
 |-- CO_ESCOLA: integer (nullable = true)
 |-- CO_MUNICIPIO_ESC: integer (nullable = true)
 |-- NO_MUNICIPIO_ESC: string (nullable = true)
 |-- CO_UF_ESC: integer (nullable = true)
 |-- SG_UF_ESC: string (nullable = true)
 |-- TP_DEPENDENCIA_ADM_ESC: integer (nullable = true)
 |-- TP_LOCALIZACAO_ESC: integer (nullable = true)
 |-- TP_SIT_FUNC_ESC: integer (nullable = true)
 |-- NU_IDADE: integer (nullable = true)
 |-- TP_SEXO: string (nullable = true)
 |-- TP_NACIONALIDADE: integer (nullable = true)
 |-- CO_MUNICIPIO_NASCIMENTO: integer (nullable = true)
 |-- NO_MUNICIPIO_NASCIMENTO: string (nu

In [8]:
# Apresentação das colunas do dataframe df_enem2015_completo

print(f'Apresentação das colunas do dataframe df_enem2015_completo - {datatime.now()}.')

df_enem2015_completo.columns[:]

['NU_INSCRICAO',
 'NU_ANO',
 'CO_MUNICIPIO_RESIDENCIA',
 'NO_MUNICIPIO_RESIDENCIA',
 'CO_UF_RESIDENCIA',
 'SG_UF_RESIDENCIA',
 'IN_ESTUDA_CLASSE_HOSPITALAR',
 'IN_TREINEIRO',
 'CO_ESCOLA',
 'CO_MUNICIPIO_ESC',
 'NO_MUNICIPIO_ESC',
 'CO_UF_ESC',
 'SG_UF_ESC',
 'TP_DEPENDENCIA_ADM_ESC',
 'TP_LOCALIZACAO_ESC',
 'TP_SIT_FUNC_ESC',
 'NU_IDADE',
 'TP_SEXO',
 'TP_NACIONALIDADE',
 'CO_MUNICIPIO_NASCIMENTO',
 'NO_MUNICIPIO_NASCIMENTO',
 'CO_UF_NASCIMENTO',
 'SG_UF_NASCIMENTO',
 'TP_ST_CONCLUSAO',
 'TP_ANO_CONCLUIU',
 'TP_ESCOLA',
 'TP_ENSINO',
 'TP_ESTADO_CIVIL',
 'TP_COR_RACA',
 'IN_BAIXA_VISAO',
 'IN_CEGUEIRA',
 'IN_SURDEZ',
 'IN_DEFICIENCIA_AUDITIVA',
 'IN_SURDO_CEGUEIRA',
 'IN_DEFICIENCIA_FISICA',
 'IN_DEFICIENCIA_MENTAL',
 'IN_DEFICIT_ATENCAO',
 'IN_DISLEXIA',
 'IN_GESTANTE',
 'IN_LACTANTE',
 'IN_IDOSO',
 'IN_DISCALCULIA',
 'IN_AUTISMO',
 'IN_VISAO_MONOCULAR',
 'IN_SABATISTA',
 'IN_OUTRA_DEF',
 'IN_SEM_RECURSO',
 'IN_NOME_SOCIAL',
 'IN_BRAILLE',
 'IN_AMPLIADA_24',
 'IN_AMPLIADA_18',
 'IN_L

In [9]:
# Descrição Dataframe df_enem2015_completo.

print('Descrição DataFrame df_enem2015_completo')
print('')

print('Descrição Idade')
df_enem2015_completo.describe('NU_IDADE').show()
print(f'{datetime.now()}.')
print('')

print('Descrição Nota da prova de Ciências da Natureza')
df_enem2015_completo.describe('NU_NOTA_CN').show()
print(f'{datetime.now()}.')
print('')

print('Descrição Nota da prova de Ciências Humanas')
df_enem2015_completo.describe('NU_NOTA_CH').show()
print(f'{datetime.now()}.')
print('')

print('Descrição Nota da prova de Linguagens e Códigos')
df_enem2015_completo.describe('NU_NOTA_LC').show()
print(f'{datetime.now()}.')
print('')

print('Descrição Nota da prova de Matemática')
df_enem2015_completo.describe('NU_NOTA_MT').show()
print(f'{datetime.now()}.')
print('')

print('Descrição Nota da prova de redação')
df_enem2015_completo.describe('NU_NOTA_REDACAO').show()
print(f'{datetime.now()}.')

Descrição DataFrame df_enem2015_completo

Descrição Idade
+-------+------------------+
|summary|          NU_IDADE|
+-------+------------------+
|  count|           7746352|
|   mean|23.004694338702915|
| stddev| 7.944599707765686|
|    min|                10|
|    max|                98|
+-------+------------------+


Descrição Nota da prova de Ciências da Natureza
+-------+-----------------+
|summary|       NU_NOTA_CN|
+-------+-----------------+
|  count|          5755091|
|   mean|478.9239503771539|
| stddev|71.56005864237716|
|    min|              0.0|
|    max|            875.2|
+-------+-----------------+


Descrição Nota da prova de Ciências Humanas
+-------+-----------------+
|summary|       NU_NOTA_CH|
+-------+-----------------+
|  count|          5755091|
|   mean|558.4878240152929|
| stddev|70.96839011437301|
|    min|              0.0|
|    max|            850.6|
+-------+-----------------+


Descrição Nota da prova de Linguagens e Códigos
+-------+------------------+
|s

In [10]:
# Descrição Dataframe df_enem2016_completo.

print('Descrição DataFrame df_enem2016_completo')
print('')

print('Descrição Idade')
df_enem2016_completo.describe('NU_IDADE').show()
print(f'{datetime.now()}.')
print('')

print('Descrição Nota da prova de Ciências da Natureza')
df_enem2016_completo.describe('NU_NOTA_CN').show()
print(f'{datetime.now()}.')
print('')

print('Descrição Nota da prova de Ciências Humanas')
df_enem2016_completo.describe('NU_NOTA_CH').show()
print(f'{datetime.now()}.')
print('')

print('Descrição Nota da prova de Linguagens e Códigos')
df_enem2016_completo.describe('NU_NOTA_LC').show()
print(f'{datetime.now()}.')
print('')

print('Descrição Nota da prova de Matemática')
df_enem2016_completo.describe('NU_NOTA_MT').show()
print(f'{datetime.now()}.')
print('')

print('Descrição Nota da prova de redação')
df_enem2016_completo.describe('NU_NOTA_REDACAO').show()
print(f'{datetime.now()}.')

Descrição DataFrame df_enem2016_completo

Descrição Idade
+-------+-----------------+
|summary|         NU_IDADE|
+-------+-----------------+
|  count|          8627265|
|   mean| 23.0310825041308|
| stddev|7.973406085745136|
|    min|               10|
|    max|               95|
+-------+-----------------+


Descrição Nota da prova de Ciências da Natureza
+-------+-----------------+
|summary|       NU_NOTA_CN|
+-------+-----------------+
|  count|          5999126|
|   mean|477.6773009101653|
| stddev|72.71845884040538|
|    min|              0.0|
|    max|            871.3|
+-------+-----------------+


Descrição Nota da prova de Ciências Humanas
+-------+-----------------+
|summary|       NU_NOTA_CH|
+-------+-----------------+
|  count|          5999126|
|   mean|534.4409175603251|
| stddev| 74.4988507782099|
|    min|              0.0|
|    max|            859.1|
+-------+-----------------+


Descrição Nota da prova de Linguagens e Códigos
+-------+-----------------+
|summary|   

In [11]:
# Descrição Dataframe df_enem2017_completo.

print('Descrição DataFrame df_enem2017_completo')
print('')

print('Descrição Idade')
df_enem2017_completo.describe('NU_IDADE').show()
print(f'{datetime.now()}.')
print('')

print('Descrição Nota da prova de Ciências da Natureza')
df_enem2017_completo.describe('NU_NOTA_CN').show()
print(f'{datetime.now()}.')
print('')

print('Descrição Nota da prova de Ciências Humanas')
df_enem2017_completo.describe('NU_NOTA_CH').show()
print(f'{datetime.now()}.')
print('')

print('Descrição Nota da prova de Linguagens e Códigos')
df_enem2017_completo.describe('NU_NOTA_LC').show()
print(f'{datetime.now()}.')
print('')

print('Descrição Nota da prova de Matemática')
df_enem2017_completo.describe('NU_NOTA_MT').show()
print(f'{datetime.now()}.')
print('')

print('Descrição Nota da prova de redação')
df_enem2017_completo.describe('NU_NOTA_REDACAO').show()
print(f'{datetime.now()}.')

Descrição DataFrame df_enem2017_completo

Descrição Idade
+-------+-----------------+
|summary|         NU_IDADE|
+-------+-----------------+
|  count|          6731240|
|   mean|22.95379261473369|
| stddev|7.728937918555041|
|    min|               10|
|    max|               90|
+-------+-----------------+


Descrição Nota da prova de Ciências da Natureza
+-------+-----------------+
|summary|       NU_NOTA_CN|
+-------+-----------------+
|  count|          4437560|
|   mean|510.8619317147265|
| stddev|72.12733741911008|
|    min|              0.0|
|    max|            885.6|
+-------+-----------------+


Descrição Nota da prova de Ciências Humanas
+-------+-----------------+
|summary|       NU_NOTA_CH|
+-------+-----------------+
|  count|          4701428|
|   mean|518.8259502219317|
| stddev|83.41395391030184|
|    min|              0.0|
|    max|            868.3|
+-------+-----------------+


Descrição Nota da prova de Linguagens e Códigos
+-------+-----------------+
|summary|   

In [12]:
# Descrição Dataframe df_enem2018_completo.

print('Descrição DataFrame df_enem2018_completo')
print('')

print('Descrição Idade')
df_enem2018_completo.describe('NU_IDADE').show()
print(f'{datetime.now()}.')
print('')

print('Descrição Nota da prova de Ciências da Natureza')
df_enem2018_completo.describe('NU_NOTA_CN').show()
print(f'{datetime.now()}.')
print('')

print('Descrição Nota da prova de Ciências Humanas')
df_enem2018_completo.describe('NU_NOTA_CH').show()
print(f'{datetime.now()}.')
print('')

print('Descrição Nota da prova de Linguagens e Códigos')
df_enem2018_completo.describe('NU_NOTA_LC').show()
print(f'{datetime.now()}.')
print('')

print('Descrição Nota da prova de Matemática')
df_enem2018_completo.describe('NU_NOTA_MT').show()
print(f'{datetime.now()}.')
print('')

print('Descrição Nota da prova de redação')
df_enem2018_completo.describe('NU_NOTA_REDACAO').show()
print(f'{datetime.now()}.')

Descrição DataFrame df_enem2018_completo

Descrição Idade
+-------+------------------+
|summary|          NU_IDADE|
+-------+------------------+
|  count|           5513650|
|   mean|22.372219854361447|
| stddev| 7.594597160609549|
|    min|                10|
|    max|                98|
+-------+------------------+


Descrição Nota da prova de Ciências da Natureza
+-------+-----------------+
|summary|       NU_NOTA_CN|
+-------+-----------------+
|  count|          3905099|
|   mean|493.6832652129926|
| stddev|  74.165826188305|
|    min|              0.0|
|    max|            869.6|
+-------+-----------------+


Descrição Nota da prova de Ciências Humanas
+-------+-----------------+
|summary|       NU_NOTA_CH|
+-------+-----------------+
|  count|          4148264|
|   mean|568.0977346186262|
| stddev|81.41996050791391|
|    min|              0.0|
|    max|            850.4|
+-------+-----------------+


Descrição Nota da prova de Linguagens e Códigos
+-------+-----------------+
|su

In [13]:
# Descrição Dataframe df_enem2019_completo.

print('Descrição DataFrame df_enem2019_completo')
print('')

print('Descrição Idade')
df_enem2019_completo.describe('NU_IDADE').show()
print(f'{datetime.now()}.')
print('')

print('Descrição Nota da prova de Ciências da Natureza')
df_enem2019_completo.describe('NU_NOTA_CN').show()
print(f'{datetime.now()}.')
print('')

print('Descrição Nota da prova de Ciências Humanas')
df_enem2019_completo.describe('NU_NOTA_CH').show()
print(f'{datetime.now()}.')
print('')

print('Descrição Nota da prova de Linguagens e Códigos')
df_enem2019_completo.describe('NU_NOTA_LC').show()
print(f'{datetime.now()}.')
print('')

print('Descrição Nota da prova de Matemática')
df_enem2019_completo.describe('NU_NOTA_MT').show()
print(f'{datetime.now()}.')
print('')

print('Descrição Nota da prova de redação')
df_enem2019_completo.describe('NU_NOTA_REDACAO').show()
print(f'{datetime.now()}.')

Descrição DataFrame df_enem2019_completo

Descrição Idade
+-------+-----------------+
|summary|         NU_IDADE|
+-------+-----------------+
|  count|          5095270|
|   mean|22.08743069552742|
| stddev|7.454970482110064|
|    min|                1|
|    max|               94|
+-------+-----------------+


Descrição Nota da prova de Ciências da Natureza
+-------+------------------+
|summary|        NU_NOTA_CN|
+-------+------------------+
|  count|           3710433|
|   mean|477.82408735045146|
| stddev| 76.09774469638398|
|    min|               0.0|
|    max|             860.9|
+-------+------------------+


Descrição Nota da prova de Ciências Humanas
+-------+-----------------+
|summary|       NU_NOTA_CH|
+-------+-----------------+
|  count|          3923145|
|   mean|507.2486632280978|
| stddev| 82.4116343330404|
|    min|              0.0|
|    max|            835.1|
+-------+-----------------+


Descrição Nota da prova de Linguagens e Códigos
+-------+-----------------+
|su

In [14]:
# Criação e seleção das colunas para realização das analises exploratórias

print(f'Criação da variável columns e seleção das colunas para realização das analises exploratórias - {datetime.now()}.')

columns = [
'NU_INSCRICAO', 
'NU_ANO', 
'NO_MUNICIPIO_RESIDENCIA', 
'CO_UF_RESIDENCIA', 
'SG_UF_RESIDENCIA',
'NU_IDADE', 
'TP_SEXO', 
'TP_ESTADO_CIVIL',
'TP_COR_RACA', 
'TP_NACIONALIDADE',
'NO_MUNICIPIO_NASCIMENTO',
'CO_UF_NASCIMENTO', 
'SG_UF_NASCIMENTO', 
'TP_ST_CONCLUSAO',
'TP_ESCOLA', 
'TP_ENSINO', 
'IN_TREINEIRO', 
'CO_UF_ESC', 
'SG_UF_ESC',
'TP_DEPENDENCIA_ADM_ESC', 
'TP_LOCALIZACAO_ESC', 
'IN_BAIXA_VISAO', 
'IN_CEGUEIRA', 
'IN_SURDEZ',
'IN_DEFICIENCIA_AUDITIVA', 
'IN_SURDO_CEGUEIRA',
'IN_DEFICIENCIA_FISICA', 
'IN_DEFICIENCIA_MENTAL',
'IN_DEFICIT_ATENCAO', 
'IN_DISLEXIA', 
'IN_DISCALCULIA', 
'IN_AUTISMO', 
'IN_VISAO_MONOCULAR',
'IN_OUTRA_DEF', 
'IN_GESTANTE', 
'IN_LACTANTE',
'IN_IDOSO', 
'IN_ESTUDA_CLASSE_HOSPITALAR', 
'IN_SEM_RECURSO', 
'IN_BRAILLE', 
'IN_AMPLIADA_24', 
'IN_AMPLIADA_18', 
'IN_LEDOR',
'IN_ACESSO', 
'IN_TRANSCRICAO', 
'IN_LIBRAS', 
'IN_LEITURA_LABIAL',
'IN_MESA_CADEIRA_RODAS', 
'IN_MESA_CADEIRA_SEPARADA',
'IN_APOIO_PERNA', 
'IN_GUIA_INTERPRETE', 
'IN_COMPUTADOR',
'IN_CADEIRA_ESPECIAL', 
'IN_CADEIRA_CANHOTO',
'IN_CADEIRA_ACOLCHOADA', 
'IN_PROVA_DEITADO', 
'IN_MOBILIARIO_OBESO',
'IN_LAMINA_OVERLAY', 
'IN_PROTETOR_AURICULAR', 
'IN_MEDIDOR_GLICOSE',
'IN_MAQUINA_BRAILE', 
'IN_SOROBAN', 
'IN_MARCA_PASSO', 
'IN_SONDA',
'IN_MEDICAMENTOS', 
'IN_SALA_INDIVIDUAL', 
'IN_SALA_ESPECIAL',
'IN_SALA_ACOMPANHANTE', 
'IN_MOBILIARIO_ESPECIFICO',
'IN_NOME_SOCIAL',
'NO_MUNICIPIO_PROVA', 
'CO_UF_PROVA', 
'SG_UF_PROVA',
'TP_PRESENCA_CN', 
'TP_PRESENCA_CH', 
'TP_PRESENCA_LC',
'TP_PRESENCA_MT', 
'NU_NOTA_CN', 
'NU_NOTA_CH', 
'NU_NOTA_LC',
'NU_NOTA_MT', 
'TP_LINGUA',
'TP_STATUS_REDACAO', 
'NU_NOTA_COMP1',
'NU_NOTA_COMP2', 
'NU_NOTA_COMP3', 
'NU_NOTA_COMP4', 
'NU_NOTA_COMP5',
'NU_NOTA_REDACAO', 
'Q001', 
'Q002', 
'Q005', 
'Q006',
'Q022',
'Q024', 
'Q025'
]


In [15]:
# No Processo abaixo, criaremos dataframes, apenas com colunas selecionadas.

print(f'Criação Dataframes Iniciado - {datetime.now()}.')
print('')

print('')
print(f'Criação Dataframe do Enem 2015 iniciada - {datetime.now()}.')
df_enem2015 = df_enem2015_completo.select(columns)
print(f'Dataframe df_enem2015 criado com sucesso - {datetime.now()}.')
print('')

print('')
print(f'Criação Dataframe do Enem 2016 iniciada - {datetime.now()}.')
df_enem2016 = df_enem2016_completo.select(columns)
print(f'Dataframe df_enem2016 criado com sucesso - {datetime.now()}.')
print('')

print('')
print(f'Criação Dataframe do Enem 2017 iniciada - {datetime.now()}.')
df_enem2017 = df_enem2017_completo.select(columns)
print(f'Dataframe df_enem2017 criado com sucesso - {datetime.now()}.')
print('')

print('')
print(f'Criação Dataframe do Enem 2018 iniciada - {datetime.now()}.')
df_enem2018 = df_enem2018_completo.select(columns)
print(f'Dataframe df_enem2018 criado com sucesso - {datetime.now()}.')
print('')

print('')
print(f'Criação Dataframe do Enem 2019 iniciada - {datetime.now()}.')
df_enem2019 = df_enem2019_completo.select(columns)
print(f'Dataframe df_enem2019 criado com sucesso - {datetime.now()}.')
print('')


Criação Dataframes Iniciado - 2020-08-20 15:18:59.764455.


Criação Dataframe do Enem 2015 iniciada - 2020-08-20 15:18:59.764455.
Dataframe df_enem2015 criado com sucesso - 2020-08-20 15:19:00.007316.


Criação Dataframe do Enem 2016 iniciada - 2020-08-20 15:19:00.007316.
Dataframe df_enem2016 criado com sucesso - 2020-08-20 15:19:00.219391.


Criação Dataframe do Enem 2017 iniciada - 2020-08-20 15:19:00.221391.
Dataframe df_enem2017 criado com sucesso - 2020-08-20 15:19:00.403803.


Criação Dataframe do Enem 2018 iniciada - 2020-08-20 15:19:00.403803.
Dataframe df_enem2018 criado com sucesso - 2020-08-20 15:19:00.576726.


Criação Dataframe do Enem 2019 iniciada - 2020-08-20 15:19:00.577691.
Dataframe df_enem2019 criado com sucesso - 2020-08-20 15:19:00.736703.



In [16]:
# Criação de um dataframe uníco, com a união de todos os dataframes criados. 

print(f'Criação Dataframe do df_enem iniciada - {datetime.now()}.')
df_enem = df_enem2015.unionAll(df_enem2016).unionAll(df_enem2017).unionAll(df_enem2018).unionAll(df_enem2019)
print(f'Dataframe df_enem criado com sucesso - {datetime.now()}.')

In [17]:
# Total de linhas do dataframe df_enem

print(f'Quantdade de Registros Dataframe df_enem - {datetime.now()}.')
df_enem.count()

33714152

In [18]:
# Apresentação das colunas do dataframe df_enem.

print(f'Apresentação das colunas do dataframe df_enem - {datatime.now()}.')

df_enem.columns[:]

['NU_INSCRICAO',
 'NU_ANO',
 'NO_MUNICIPIO_RESIDENCIA',
 'CO_UF_RESIDENCIA',
 'SG_UF_RESIDENCIA',
 'NU_IDADE',
 'TP_SEXO',
 'TP_ESTADO_CIVIL',
 'TP_COR_RACA',
 'TP_NACIONALIDADE',
 'NO_MUNICIPIO_NASCIMENTO',
 'CO_UF_NASCIMENTO',
 'SG_UF_NASCIMENTO',
 'TP_ST_CONCLUSAO',
 'TP_ESCOLA',
 'TP_ENSINO',
 'IN_TREINEIRO',
 'CO_UF_ESC',
 'SG_UF_ESC',
 'TP_DEPENDENCIA_ADM_ESC',
 'TP_LOCALIZACAO_ESC',
 'IN_BAIXA_VISAO',
 'IN_CEGUEIRA',
 'IN_SURDEZ',
 'IN_DEFICIENCIA_AUDITIVA',
 'IN_SURDO_CEGUEIRA',
 'IN_DEFICIENCIA_FISICA',
 'IN_DEFICIENCIA_MENTAL',
 'IN_DEFICIT_ATENCAO',
 'IN_DISLEXIA',
 'IN_DISCALCULIA',
 'IN_AUTISMO',
 'IN_VISAO_MONOCULAR',
 'IN_OUTRA_DEF',
 'IN_GESTANTE',
 'IN_LACTANTE',
 'IN_IDOSO',
 'IN_ESTUDA_CLASSE_HOSPITALAR',
 'IN_SEM_RECURSO',
 'IN_BRAILLE',
 'IN_AMPLIADA_24',
 'IN_AMPLIADA_18',
 'IN_LEDOR',
 'IN_ACESSO',
 'IN_TRANSCRICAO',
 'IN_LIBRAS',
 'IN_LEITURA_LABIAL',
 'IN_MESA_CADEIRA_RODAS',
 'IN_MESA_CADEIRA_SEPARADA',
 'IN_APOIO_PERNA',
 'IN_GUIA_INTERPRETE',
 'IN_COMPUTADOR

In [19]:
# Apresentação da estutura do dataframe df_enem

print(f'Apresentação da estutura do dataframe df_enem - {datatime.now()}.')

df_enem.printSchema()

root
 |-- NU_INSCRICAO: long (nullable = true)
 |-- NU_ANO: integer (nullable = true)
 |-- NO_MUNICIPIO_RESIDENCIA: string (nullable = true)
 |-- CO_UF_RESIDENCIA: integer (nullable = true)
 |-- SG_UF_RESIDENCIA: string (nullable = true)
 |-- NU_IDADE: integer (nullable = true)
 |-- TP_SEXO: string (nullable = true)
 |-- TP_ESTADO_CIVIL: integer (nullable = true)
 |-- TP_COR_RACA: integer (nullable = true)
 |-- TP_NACIONALIDADE: integer (nullable = true)
 |-- NO_MUNICIPIO_NASCIMENTO: string (nullable = true)
 |-- CO_UF_NASCIMENTO: integer (nullable = true)
 |-- SG_UF_NASCIMENTO: string (nullable = true)
 |-- TP_ST_CONCLUSAO: integer (nullable = true)
 |-- TP_ESCOLA: integer (nullable = true)
 |-- TP_ENSINO: integer (nullable = true)
 |-- IN_TREINEIRO: integer (nullable = true)
 |-- CO_UF_ESC: integer (nullable = true)
 |-- SG_UF_ESC: string (nullable = true)
 |-- TP_DEPENDENCIA_ADM_ESC: integer (nullable = true)
 |-- TP_LOCALIZACAO_ESC: integer (nullable = true)
 |-- IN_BAIXA_VISAO: in

In [20]:
# Descrição Dataframe df_enem.

print('Descrição DataFrame df_enem (Dataframe com os anos 2015, 2016, 2017, 2018 e 2019).')
print('')

print('Descrição Idade')
df_enem.describe('NU_IDADE').show()
print(f'{datetime.now()}.')
print('')

print('Descrição Nota da prova de Ciências da Natureza')
df_enem.describe('NU_NOTA_CN').show()
print(f'{datetime.now()}.')
print('')

print('Descrição Nota da prova de Ciências Humanas')
df_enem.describe('NU_NOTA_CH').show()
print(f'{datetime.now()}.')
print('')

print('Descrição Nota da prova de Linguagens e Códigos')
df_enem.describe('NU_NOTA_LC').show()
print(f'{datetime.now()}.')
print('')

print('Descrição Nota da prova de Matemática')
df_enem.describe('NU_NOTA_MT').show()
print(f'{datetime.now()}.')
print('')

print('Descrição Nota da prova de redação')
df_enem.describe('NU_NOTA_REDACAO').show()
print(f'{datetime.now()}.')

Descrição DataFrame df_enem (Dataframe com os anos 2015, 2016, 2017, 2018 e 2019).

Descrição Idade
+-------+------------------+
|summary|          NU_IDADE|
+-------+------------------+
|  count|          33713777|
|   mean|22.759218345663257|
| stddev| 7.788652341876025|
|    min|                 1|
|    max|                98|
+-------+------------------+


Descrição Nota da prova de Ciências da Natureza
+-------+-----------------+
|summary|       NU_NOTA_CN|
+-------+-----------------+
|  count|         23807309|
|   mean|486.8124319678461|
| stddev|74.22313690738514|
|    min|              0.0|
|    max|            885.6|
+-------+-----------------+


Descrição Nota da prova de Ciências Humanas
+-------+-----------------+
|summary|       NU_NOTA_CH|
+-------+-----------------+
|  count|         24527054|
|   mean|538.4331498189712|
| stddev|80.97454950674937|
|    min|              0.0|
|    max|            868.3|
+-------+-----------------+


Descrição Nota da prova de Linguagens

In [21]:
# Criar tabela temporaria e usar sql para consulta

print(f'Apresentação do processo de criação de uma tabela temporária para utilização do SQL - {datetime.now()}.')
df_enem.createOrReplaceTempView('enemtb')
teste = sqlContext.sql('SELECT NU_INSCRICAO FROM enemtb').show()
print(f'{datetime.now()}.')

+------------+
|NU_INSCRICAO|
+------------+
|150000000376|
|150000003053|
|150000001161|
|150000001335|
|150000001219|
|150000002884|
|150000002510|
|150000000658|
|150000000434|
|150000000216|
|150000000274|
|150000000217|
|150000000164|
|150000000165|
|150000000218|
|150000000166|
|150000000219|
|150000000275|
|150000000220|
|150000000167|
+------------+
only showing top 20 rows



In [22]:
# Criação das colunas de classificação do dataframe df_enem_analise. 

print(f'Criação da coluna SEXO - {datetime.now()}.')

df_enem_sexo = df_enem.withColumn('SEXO', expr("case when TP_SEXO = 'M' then 'Masculino' " + "when TP_SEXO = 'F' then 'Feminino' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

In [23]:
print(f'Criação da coluna ESTADO_CIVIL - {datetime.now()}.')

df_enem_estado_civil = df_enem_sexo.withColumn('ESTADO_CIVIL', expr("case when TP_ESTADO_CIVIL = 0 then 'Solteiro' " + "when TP_ESTADO_CIVIL = 1 then 'Casado(a)/Mora com companheiro(a)' " + "when TP_ESTADO_CIVIL = 2 then 'Divorciado(a)/Desquitado(a)/Separado(a)' " + "when TP_ESTADO_CIVIL = 3 then 'Viúvo(a)' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

In [24]:
print(f'Criação da coluna RACA - {datetime.now()}.')

df_enem_raca = df_enem_estado_civil.withColumn('RACA', expr("case when TP_NACIONALIDADE = 0 then 'Não declarado' " + "when TP_NACIONALIDADE = 1 then 'Branca' " + "when TP_NACIONALIDADE = 2 then 'Preta' " + "when TP_NACIONALIDADE = 3 then 'Parda' " + "when TP_NACIONALIDADE = 4 then 'Amarela' " + "when TP_NACIONALIDADE = 5 then 'Indígena ' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

In [25]:
print(f'Criação da coluna NACIONALIDADE - {datetime.now()}.')

df_enem_nacionalidade = df_enem_raca.withColumn('NACIONALIDADE', expr("case when TP_NACIONALIDADE = 0 then 'Não informado' " + "when TP_NACIONALIDADE = 1 then 'Brasileiro(a)' " + "when TP_NACIONALIDADE = 2 then 'Brasileiro(a) Naturalizado(a)' " + "when TP_NACIONALIDADE = 3 then 'Estrangeiro(a)' " + "when TP_NACIONALIDADE = 4 then 'Brasileiro(a) Nato(a), nascido(a) no exterior' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

In [26]:
print(f'Criação da coluna SITUACAO_CONCLUSAO_ENSINO_MEDIO - {datetime.now()}.')

df_enem_st_conclusao = df_enem_nacionalidade.withColumn('SITUACAO_CONCLUSAO_ENSINO_MEDIO', expr("case when TP_ST_CONCLUSAO = 1 then 'Já concluí o Ensino Médio' " + "when TP_ST_CONCLUSAO = 2 then 'Estou cursando e concluirei o Ensino Médio no ano' " + "when TP_ST_CONCLUSAO = 3 then 'Estou cursando e concluirei o Ensino Médio após este ano' " + "when TP_ST_CONCLUSAO = 4 then 'Não concluí e não estou cursando o Ensino Médio' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

In [27]:
print(f'Criação da coluna TIPO_ESCOLA - {datetime.now()}.')

df_enem_tp_escola = df_enem_st_conclusao.withColumn('TIPO_ESCOLA', expr("case when TP_ESCOLA = 1 then 'Não Respondeu' " + "when TP_ESCOLA = 2 then 'Pública' " + "when TP_ESCOLA = 3 then 'Privada' " + "when TP_ESCOLA = 4 then 'Exterior' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

In [28]:
print(f'Criação da coluna TIPO_ENSINO - {datetime.now()}.')

df_enem_tp_ensino = df_enem_tp_escola.withColumn('TIPO_ENSINO', expr("case when TP_ENSINO = 1 then 'Ensino Regular' " + "when TP_ENSINO = 2 then 'Educação Especial - Modalidade Substitutiva' " + "when TP_ENSINO = 3 then 'Educação de Jovens e Adultos' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

In [29]:
print(f'Criação da coluna TREINEIRO - {datetime.now()}.')

df_enem_in_treineiro = df_enem_tp_ensino.withColumn('TREINEIRO', expr("case when IN_TREINEIRO = 0 then 'Não' " + "when IN_TREINEIRO = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

In [30]:
print(f'Criação da coluna DEPENDENCIA_ADMINISTRATIVA_ESCOLA - {datetime.now()}.')

df_enem_tp_dependencia_adm_esc = df_enem_in_treineiro.withColumn('DEPENDENCIA_ADMINISTRATIVA_ESCOLA', expr("case when TP_DEPENDENCIA_ADM_ESC = 1 then 'Federal' " + "when TP_DEPENDENCIA_ADM_ESC = 2 then 'Estadual' " + "when TP_DEPENDENCIA_ADM_ESC = 3 then 'Municipal' " + "when TP_DEPENDENCIA_ADM_ESC = 4 then 'Privada' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

In [31]:
print(f'Criação da coluna LOCALIZACAO_ESCOLA - {datetime.now()}.')

df_enem_tp_localizacao_esc = df_enem_tp_dependencia_adm_esc.withColumn('LOCALIZACAO_ESCOLA', expr("case when TP_LOCALIZACAO_ESC = 1 then 'Urbana' " + "when TP_LOCALIZACAO_ESC = 2 then 'Rural' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

In [32]:
print(f'Criação da coluna BAIXA_VISAO - {datetime.now()}.')

df_enem_in_baixa_visao = df_enem_tp_localizacao_esc.withColumn('BAIXA_VISAO', expr("case when IN_BAIXA_VISAO = 0 then 'Não' " + "when IN_BAIXA_VISAO = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

In [33]:
print(f'Criação da coluna CEGUEIRA - {datetime.now()}.')

df_enem_in_cegueira = df_enem_in_baixa_visao.withColumn('CEGUEIRA', expr("case when IN_CEGUEIRA = 0 then 'Não' " + "when IN_CEGUEIRA = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')


In [34]:
print(f'Criação da coluna SURDEZ - {datetime.now()}.')

df_enem_in_surdez = df_enem_in_cegueira.withColumn('SURDEZ', expr("case when IN_SURDEZ = 0 then 'Não' " + "when IN_SURDEZ = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

In [35]:
print(f'Criação da coluna DEFICIENCIA_AUDITIVA - {datetime.now()}.')

df_enem_in_deficiencia_auditiva = df_enem_in_surdez.withColumn('DEFICIENCIA_AUDITIVA', expr("case when IN_DEFICIENCIA_AUDITIVA = 0 then 'Não' " + "when IN_DEFICIENCIA_AUDITIVA = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

In [36]:
print(f'Criação da coluna SURDO_CEGUEIRA - {datetime.now()}.')

df_enem_in_surdo_cegueira = df_enem_in_deficiencia_auditiva.withColumn('SURDO_CEGUEIRA', expr("case when IN_SURDO_CEGUEIRA = 0 then 'Não' " + "when IN_SURDO_CEGUEIRA = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

In [38]:
print(f'Criação da coluna DEFICIENCIA_FISICA - {datetime.now()}.')

df_enem_in_deficiencia_fisica = df_enem_in_surdo_cegueira.withColumn('DEFICIENCIA_FISICA', expr("case when IN_DEFICIENCIA_FISICA = 0 then 'Não' " + "when IN_DEFICIENCIA_FISICA = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

In [39]:
print(f'Criação da coluna DEFICIENCIA_MENTAL - {datetime.now()}.')

df_enem_in_deficiencia_mental = df_enem_in_deficiencia_fisica.withColumn('DEFICIENCIA_MENTAL', expr("case when IN_DEFICIENCIA_MENTAL = 0 then 'Não' " + "when IN_DEFICIENCIA_MENTAL = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

In [40]:
print(f'Criação da coluna DEFICIT_ATENCAO - {datetime.now()}.')

df_enem_in_deficit_atencao = df_enem_in_deficiencia_mental.withColumn('DEFICIT_ATENCAO', expr("case when IN_DEFICIT_ATENCAO = 0 then 'Não' " + "when IN_DEFICIT_ATENCAO = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

In [41]:
print(f'Criação da coluna DISLEXIA - {datetime.now()}.')

df_enem_in_dislexia = df_enem_in_deficit_atencao.withColumn('DISLEXIA', expr("case when IN_DISLEXIA = 0 then 'Não' " + "when IN_DISLEXIA = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

In [42]:
print(f'Criação da coluna DISCALCULIA - {datetime.now()}.')

df_enem_in_discalculia = df_enem_in_dislexia.withColumn('DISCALCULIA', expr("case when IN_DISCALCULIA = 0 then 'Não' " + "when IN_DISCALCULIA = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

In [43]:
print(f'Criação da coluna AUTISMO - {datetime.now()}.')

df_enem_in_autismo = df_enem_in_discalculia.withColumn('AUTISMO', expr("case when IN_AUTISMO = 0 then 'Não' " + "when IN_AUTISMO = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

In [44]:
print(f'Criação da coluna VISAO_MONOCULAR - {datetime.now()}.')

df_enem_in_visao_monocular = df_enem_in_autismo.withColumn('VISAO_MONOCULAR', expr("case when IN_VISAO_MONOCULAR = 0 then 'Não' " + "when IN_VISAO_MONOCULAR = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

In [45]:
print(f'Criação da coluna OUTRA_DEF - {datetime.now()}.')

df_enem_in_outra_def = df_enem_in_visao_monocular.withColumn('OUTRA_DEF', expr("case when IN_OUTRA_DEF = 0 then 'Não' " + "when IN_OUTRA_DEF = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

In [46]:
print(f'Criação da coluna GESTANTE - {datetime.now()}.')

df_enem_in_gestante = df_enem_in_outra_def.withColumn('GESTANTE', expr("case when IN_GESTANTE = 0 then 'Não' " + "when IN_GESTANTE = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

In [47]:
print(f'Criação da coluna LACTANTE - {datetime.now()}.')

df_enem_in_lactante = df_enem_in_gestante.withColumn('LACTANTE', expr("case when IN_LACTANTE = 0 then 'Não' " + "when IN_LACTANTE = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

In [48]:
print(f'Criação da coluna IDOSO - {datetime.now()}.')

df_enem_in_idoso = df_enem_in_lactante.withColumn('IDOSO', expr("case when IN_IDOSO = 0 then 'Não' " + "when IN_IDOSO = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

In [49]:
print(f'Criação da coluna ESTUDA_CLASSE_HOSPITALAR - {datetime.now()}.')

df_enem_in_estuda_classe_hospitalar = df_enem_in_idoso.withColumn('ESTUDA_CLASSE_HOSPITALAR', expr("case when IN_ESTUDA_CLASSE_HOSPITALAR = 0 then 'Não' " + "when IN_ESTUDA_CLASSE_HOSPITALAR = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

In [50]:
print(f'Criação da coluna SEM_RECURSO - {datetime.now()}.')

df_enem_in_sem_recurso = df_enem_in_estuda_classe_hospitalar.withColumn('SEM_RECURSO', expr("case when IN_SEM_RECURSO = 0 then 'Não' " + "when IN_SEM_RECURSO = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

In [51]:
print(f'Criação da coluna BRAILLE - {datetime.now()}.')

df_enem_in_braille = df_enem_in_sem_recurso.withColumn('BRAILLE', expr("case when IN_BRAILLE = 0 then 'Não' " + "when IN_BRAILLE = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

In [52]:
print(f'Criação da coluna AMPLIADA_24 - {datetime.now()}.')

df_enem_in_ampliada_24 = df_enem_in_braille.withColumn('AMPLIADA_24', expr("case when IN_AMPLIADA_24 = 0 then 'Não' " + "when IN_AMPLIADA_24 = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

In [53]:
print(f'Criação da coluna AMPLIADA_18 - {datetime.now()}.')

df_enem_in_ampliada_18 = df_enem_in_ampliada_24.withColumn('AMPLIADA_18', expr("case when IN_AMPLIADA_18 = 0 then 'Não' " + "when IN_AMPLIADA_18 = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

In [54]:
print(f'Criação da coluna LEDOR - {datetime.now()}.')

df_enem_in_ledor = df_enem_in_ampliada_18.withColumn('LEDOR', expr("case when IN_LEDOR = 0 then 'Não' " + "when IN_LEDOR = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

In [55]:
print(f'Criação da coluna ACESSO - {datetime.now()}.')

df_enem_in_acesso = df_enem_in_ledor.withColumn('ACESSO', expr("case when IN_ACESSO = 0 then 'Não' " + "when IN_ACESSO = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

In [56]:
print(f'Criação da coluna TRANSCRICAO - {datetime.now()}.')

df_enem_in_transcricao = df_enem_in_acesso.withColumn('TRANSCRICAO', expr("case when IN_TRANSCRICAO = 0 then 'Não' " + "when IN_TRANSCRICAO = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

In [57]:
print(f'Criação da coluna LIBRAS - {datetime.now()}.')

df_enem_in_libras = df_enem_in_transcricao.withColumn('LIBRAS', expr("case when IN_LIBRAS = 0 then 'Não' " + "when IN_LIBRAS = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

In [58]:
print(f'Criação da coluna LEITURA_LABIAL - {datetime.now()}.')

df_enem_in_leitura_labial = df_enem_in_libras.withColumn('LEITURA_LABIAL', expr("case when IN_LEITURA_LABIAL = 0 then 'Não' " + "when IN_LEITURA_LABIAL = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

In [59]:
print(f'Criação da coluna MESA_CADEIRA_RODAS - {datetime.now()}.')

df_enem_in_mesa_cadeira_rodas = df_enem_in_leitura_labial.withColumn('MESA_CADEIRA_RODAS', expr("case when IN_MESA_CADEIRA_RODAS = 0 then 'Não' " + "when IN_MESA_CADEIRA_RODAS = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

In [60]:
print(f'Criação da coluna MESA_CADEIRA_SEPARADA - {datetime.now()}.')

df_enem_in_mesa_cadeira_separada = df_enem_in_mesa_cadeira_rodas.withColumn('MESA_CADEIRA_SEPARADA', expr("case when IN_MESA_CADEIRA_SEPARADA = 0 then 'Não' " + "when IN_MESA_CADEIRA_SEPARADA = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

In [61]:
print(f'Criação da coluna APOIO_PERNA - {datetime.now()}.')

df_enem_in_apoio_perna = df_enem_in_mesa_cadeira_separada.withColumn('APOIO_PERNA', expr("case when IN_APOIO_PERNA = 0 then 'Não' " + "when IN_APOIO_PERNA = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

In [62]:
print(f'Criação da coluna GUIA_INTERPRETE - {datetime.now()}.')

df_enem_in_guia_interprete = df_enem_in_apoio_perna.withColumn('GUIA_INTERPRETE', expr("case when IN_GUIA_INTERPRETE = 0 then 'Não' " + "when IN_GUIA_INTERPRETE = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

In [63]:
print(f'Criação da coluna COMPUTADOR - {datetime.now()}.')

df_enem_in_computador = df_enem_in_guia_interprete.withColumn('COMPUTADOR', expr("case when IN_COMPUTADOR = 0 then 'Não' " + "when IN_COMPUTADOR = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

In [64]:
print(f'Criação da coluna CADEIRA_ESPECIAL - {datetime.now()}.')

df_enem_in_cadeira_especial = df_enem_in_computador.withColumn('CADEIRA_ESPECIAL', expr("case when IN_CADEIRA_ESPECIAL = 0 then 'Não' " + "when IN_CADEIRA_ESPECIAL = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

In [65]:
print(f'Criação da coluna CADEIRA_CANHOTO - {datetime.now()}.')

df_enem_in_cadeira_canhoto = df_enem_in_cadeira_especial.withColumn('CADEIRA_CANHOTO', expr("case when IN_CADEIRA_CANHOTO = 0 then 'Não' " + "when IN_CADEIRA_CANHOTO = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

In [66]:
print(f'Criação da coluna CADEIRA_ACOLCHOADA - {datetime.now()}.')

df_enem_in_cadeira_acolchoada = df_enem_in_cadeira_canhoto.withColumn('CADEIRA_ACOLCHOADA', expr("case when IN_CADEIRA_ACOLCHOADA = 0 then 'Não' " + "when IN_CADEIRA_ACOLCHOADA = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

In [67]:
print(f'Criação da coluna PROVA_DEITADO - {datetime.now()}.')

df_enem_in_prova_deitado = df_enem_in_cadeira_acolchoada.withColumn('PROVA_DEITADO', expr("case when IN_PROVA_DEITADO = 0 then 'Não' " + "when IN_PROVA_DEITADO = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

In [68]:
print(f'Criação da coluna MOBILIARIO_OBESO - {datetime.now()}.')

df_enem_in_mobiliario_obeso = df_enem_in_prova_deitado.withColumn('MOBILIARIO_OBESO', expr("case when IN_MOBILIARIO_OBESO = 0 then 'Não' " + "when IN_MOBILIARIO_OBESO = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

In [69]:
print(f'Criação da coluna LAMINA_OVERLAY - {datetime.now()}.')

df_enem_in_lamina_overlay = df_enem_in_mobiliario_obeso.withColumn('LAMINA_OVERLAY', expr("case when IN_LAMINA_OVERLAY = 0 then 'Não' " + "when IN_LAMINA_OVERLAY = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

In [70]:
print(f'Criação da coluna PROTETOR_AURICULAR - {datetime.now()}.')

df_enem_in_protetor_auricular = df_enem_in_lamina_overlay.withColumn('PROTETOR_AURICULAR', expr("case when IN_PROTETOR_AURICULAR = 0 then 'Não' " + "when IN_PROTETOR_AURICULAR = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

In [71]:
print(f'Criação da coluna MEDIDOR_GLICOSE - {datetime.now()}.')

df_enem_in_medidor_glicose = df_enem_in_protetor_auricular.withColumn('MEDIDOR_GLICOSE', expr("case when IN_MEDIDOR_GLICOSE = 0 then 'Não' " + "when IN_MEDIDOR_GLICOSE = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

In [72]:
print(f'Criação da coluna MAQUINA_BRAILE - {datetime.now()}.')

df_enem_in_maquina_braile = df_enem_in_medidor_glicose.withColumn('MAQUINA_BRAILE', expr("case when IN_MAQUINA_BRAILE = 0 then 'Não' " + "when IN_MAQUINA_BRAILE = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

In [73]:
print(f'Criação da coluna SOROBAN - {datetime.now()}.')

df_enem_in_soroban = df_enem_in_maquina_braile.withColumn('SOROBAN', expr("case when IN_SOROBAN = 0 then 'Não' " + "when IN_SOROBAN = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

In [74]:
print(f'Criação da coluna MARCA_PASSO - {datetime.now()}.')

df_enem_in_marca_passo = df_enem_in_soroban.withColumn('MARCA_PASSO', expr("case when IN_MARCA_PASSO = 0 then 'Não' " + "when IN_MARCA_PASSO = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

In [75]:
print(f'Criação da coluna SONDA - {datetime.now()}.')

df_enem_in_sonda = df_enem_in_marca_passo.withColumn('SONDA', expr("case when IN_SONDA = 0 then 'Não' " + "when IN_SONDA = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

In [76]:
print(f'Criação da coluna MEDICAMENTOS - {datetime.now()}.')

df_enem_in_medicamentos = df_enem_in_sonda.withColumn('MEDICAMENTOS', expr("case when IN_MEDICAMENTOS = 0 then 'Não' " + "when IN_MEDICAMENTOS = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

In [77]:
print(f'Criação da coluna SALA_INDIVIDUAL - {datetime.now()}.')

df_enem_in_sala_individual = df_enem_in_medicamentos.withColumn('SALA_INDIVIDUAL', expr("case when IN_SALA_INDIVIDUAL = 0 then 'Não' " + "when IN_SALA_INDIVIDUAL = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

In [78]:
print(f'Criação da coluna SALA_ESPECIAL - {datetime.now()}.')

df_enem_in_sala_especial = df_enem_in_sala_individual.withColumn('SALA_ESPECIAL', expr("case when IN_SALA_ESPECIAL = 0 then 'Não' " + "when IN_SALA_ESPECIAL = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

In [79]:
print(f'Criação da coluna SALA_ACOMPANHANTE - {datetime.now()}.')

df_enem_in_sala_acompanhante = df_enem_in_sala_especial.withColumn('SALA_ACOMPANHANTE', expr("case when IN_SALA_ACOMPANHANTE = 0 then 'Não' " + "when IN_SALA_ACOMPANHANTE = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

In [80]:
print(f'Criação da coluna MOBILIARIO_ESPECIFICO - {datetime.now()}.')

df_enem_in_mobiliario_especifico = df_enem_in_sala_acompanhante.withColumn('MOBILIARIO_ESPECIFICO', expr("case when IN_MOBILIARIO_ESPECIFICO = 0 then 'Não' " + "when IN_MOBILIARIO_ESPECIFICO = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

In [81]:
print(f'Criação da coluna NOME_SOCIAL - {datetime.now()}.')

df_enem_in_nome_social = df_enem_in_mobiliario_especifico.withColumn('NOME_SOCIAL', expr("case when IN_NOME_SOCIAL = 0 then 'Não' " + "when IN_NOME_SOCIAL = 1 then 'Sim' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

In [82]:
print(f'Criação da coluna PRESENCA_CIENCIAS_NATUREZA - {datetime.now()}.')

df_enem_tp_presenca_cn = df_enem_in_nome_social.withColumn('PRESENCA_CIENCIAS_NATUREZA', expr("case when TP_PRESENCA_CN = 0 then 'Faltou à prova' " + "when TP_PRESENCA_CN = 1 then 'Presente na prova' " + "when TP_PRESENCA_CN = 2 then 'Eliminado na prova' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

In [83]:
print(f'Criação da coluna PRESENCA_CIENCIAS_HUMANAS - {datetime.now()}.')

df_enem_tp_presenca_ch = df_enem_tp_presenca_cn.withColumn('PRESENCA_CIENCIAS_HUMANAS', expr("case when TP_PRESENCA_CH = 0 then 'Faltou à prova' " + "when TP_PRESENCA_CH = 1 then 'Presente na prova' " + "when TP_PRESENCA_CH = 2 then 'Eliminado na prova' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

In [84]:
print(f'Criação da coluna PRESENCA_LINGUAGENS_CODIGOS - {datetime.now()}.')

df_enem_tp_presenca_lc = df_enem_tp_presenca_ch.withColumn('PRESENCA_LINGUAGENS_CODIGOS', expr("case when TP_PRESENCA_LC = 0 then 'Faltou à prova' " + "when TP_PRESENCA_LC = 1 then 'Presente na prova' " + "when TP_PRESENCA_LC = 2 then 'Eliminado na prova' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

In [85]:
print(f'Criação da coluna PRESENCA_MATEMATICA - {datetime.now()}.')

df_enem_tp_presenca_mt = df_enem_tp_presenca_lc.withColumn('PRESENCA_MATEMATICA', expr("case when TP_PRESENCA_MT = 0 then 'Faltou à prova' " + "when TP_PRESENCA_MT = 1 then 'Presente na prova' " + "when TP_PRESENCA_MT = 2 then 'Eliminado na prova' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

In [86]:
print(f'Criação da coluna LINGUA - {datetime.now()}.')

df_enem_tp_lingua = df_enem_tp_presenca_mt.withColumn('LINGUA', expr("case when TP_LINGUA = 0 then 'Inglês' " + "when TP_LINGUA = 1 then 'Espanhol' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

In [87]:
print(f'Criação da coluna STATUS_REDACAO - {datetime.now()}.')

df_enem_tp_status_redacao = df_enem_tp_lingua.withColumn('STATUS_REDACAO', expr("case when TP_STATUS_REDACAO = 1 then 'Sem problemas' " + "when TP_STATUS_REDACAO = 2 then 'Anulada' " + "when TP_STATUS_REDACAO = 3 then 'Cópia Texto Motivador' " + "when TP_STATUS_REDACAO = 4 then 'Em Branco' " + "when TP_STATUS_REDACAO = 6 then 'Fuga ao tema' " + "when TP_STATUS_REDACAO = 7 then 'Não atendimento ao tipo textual' " + "when TP_STATUS_REDACAO = 8 then 'Texto insuficiente' " + "when TP_STATUS_REDACAO = 9 then 'Parte desconectada' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

In [88]:
print(f'Criação da coluna PAI_ESTUDOU - {datetime.now()}.')

df_enem_q001 = df_enem_tp_status_redacao.withColumn('PAI_ESTUDOU', expr("case when Q001 = 'A' then 'Nunca estudou.' " + "when Q001 = 'B' then 'Não completou a 4ª série/5º ano do Ensino Fundamental.' " + "when Q001 = 'C' then 'Completou a 4ª série/5º ano, mas não completou a 8ª série/9º ano do Ensino Fundamental.' " + "when Q001 = 'D' then 'Completou a 8ª série/9º ano do Ensino Fundamental, mas não completou o Ensino Médio.' " + "when Q001 = 'E' then 'Completou o Ensino Médio, mas não completou a Faculdade.' " + "when Q001 = 'F' then 'Completou a Faculdade, mas não completou a Pós-graduação.' " + "when Q001 = 'G' then 'Completou a Pós-graduação.' " + "when Q001 = 'H' then 'Não sei.' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

In [89]:
print(f'Criação da coluna MAE_ESTUDOU - {datetime.now()}.')

df_enem_q002 = df_enem_q001.withColumn('MAE_ESTUDOU', expr("case when Q002 = 'A' then 'Nunca estudou.' " + "when Q002 = 'B' then 'Não completou a 4ª série/5º ano do Ensino Fundamental.' " + "when Q002 = 'C' then 'Completou a 4ª série/5º ano, mas não completou a 8ª série/9º ano do Ensino Fundamental.' " + "when Q002 = 'D' then 'Completou a 8ª série/9º ano do Ensino Fundamental, mas não completou o Ensino Médio.' " + "when Q002 = 'E' then 'Completou o Ensino Médio, mas não completou a Faculdade.' " + "when Q002 = 'F' then 'Completou a Faculdade, mas não completou a Pós-graduação.' " + "when Q002 = 'G' then 'Completou a Pós-graduação.' " + "when Q002 = 'H' then 'Não sei.' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

In [90]:
print(f'Criação da coluna QTDE_MORADOR_RESIDENCIA - {datetime.now()}.')

df_enem_q005 = df_enem_q002.withColumn('QTDE_MORADOR_RESIDENCIA', expr("case when Q005 = 1 then '1, pois moro sozinho(a).' " + "when Q005 = 2 then '2' " + "when Q005 = 3 then '3' " + "when Q005 = 4 then '4' " + "when Q005 = 5 then '5' " + "when Q005 = 6 then '6' " + "when Q005 = 7 then '7' " + "when Q005 = 8 then '8' " + "when Q005 = 9 then '9' " + "when Q005 = 10 then '10' " + "when Q005 = 11 then '11' " + "when Q005 = 12 then '12' " + "when Q005 = 13 then '13' " + "when Q005 = 14 then '14' " + "when Q005 = 15 then '15' " + "when Q005 = 16 then '16' " + "when Q005 = 17 then '17' " + "when Q005 = 18 then '18' " + "when Q005 = 19 then '19' " + "when Q005 = 20 then '20' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

In [91]:
print(f'Criação da coluna RENDA_MENSAL_FAMILIAR - {datetime.now()}.')

df_enem_q006 = df_enem_q005.withColumn('RENDA_MENSAL_FAMILIAR', expr("case when Q006 = 'A' then 'Nenhuma Renda.' " + "when Q006 = 'B' then 'Até 1,0 salário mínimo.' " + "when Q006 = 'C' then 'Acima de 1,0 salário mínimo até 1,5.' " + "when Q006 = 'D' then 'Acima de 1,5 salário mínimo até 2,0 salários mínimos.' " + "when Q006 = 'E' then 'Acima de 2,0 salários mínimos até 2,5 salários mínimos.' " + "when Q006 = 'F' then 'Acima de 2,5 salários mínimos até 3,0 salários mínimos.' " + "when Q006 = 'G' then 'Acima de 3,0 salários mínimos até 4,0 salários mínimos.' " + "when Q006 = 'H' then 'Acima de 4,0 salários mínimos até 5,0 salários mínimos.' " + "when Q006 = 'I' then 'Acima de 5,0 salários mínimos até 6,0 salários mínimos.' " + "when Q006 = 'J' then 'Acima de 6,0 salários mínimos até 7,0 salários mínimos.' " + "when Q006 = 'K' then 'Acima de 7,0 salários mínimos até 8,0 salários mínimos.' " + "when Q006 = 'L' then 'Acima de 8,0 salários mínimos até 9,0 salários mínimos.' " + "when Q006 = 'M' then 'Acima de 9,0 salários mínimos até 10,0 salários mínimos.' " + "when Q006 = 'N' then 'Acima de 10,0 salários mínimos até 12,0 salários mínimos.' " + "when Q006 = 'O' then 'Acima de 12,0 salários mínimos até 15,0 salários mínimos.' " + "when Q006 = 'P' then 'Acima de 15,0 salários mínimos até 20,0 salários mínimos.' " + "when Q006 = 'Q' then 'Acima de 20 salários mínimos.' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

In [92]:
print(f'Criação da coluna RESIDENCIA_CELULAR - {datetime.now()}.')

df_enem_q022 = df_enem_q006.withColumn('RESIDENCIA_CELULAR', expr("case when Q022 = 'A' then 'Não.' " + "when Q022 = 'B' then 'Sim, um.' " + "when Q022 = 'C' then 'Sim, dois.' " + "when Q022 = 'D' then 'Sim, três.' " + "when Q022 = 'E' then 'Sim, quatro ou mais.' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

In [93]:
print(f'Criação da coluna RESIDENCIA_COMPUTADOR - {datetime.now()}.')

df_enem_q024 = df_enem_q022.withColumn('RESIDENCIA_COMPUTADOR', expr("case when Q024 = 'A' then 'Não.' " + "when Q024 = 'B' then 'Sim, um.' " + "when Q024 = 'C' then 'Sim, dois.' " + "when Q024 = 'D' then 'Sim, três.' " + "when Q024 = 'E' then 'Sim, quatro ou mais.' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

In [94]:
print(f'Criação da coluna RESIDENCIA_INTERNET - {datetime.now()}.')

df_enem_q025 = df_enem_q024.withColumn('RESIDENCIA_INTERNET', expr("case when Q025 = 'A' then 'Não.' " + "when Q025 = 'B' then 'Sim.' " + "else '' end"))

print(f'Criação da coluna com sucesso - {datetime.now()}.')

In [95]:
# Criação do dataframe df_enem_analise para análise exploratória.

print(f'Criação do Dataframe para análise exploratória df_enem_analise - {datetime.now()}.')

df_enem_analise = df_enem_q025

print(f'Dataframe df_enem_analise criado com sucesso - {datetime.now()}.')

In [96]:
# Apresentação da estutura do dataframe df_enem_analise

print(f'Apresentação da estutura do dataframe df_enem_analise - {datatime.now()}.')

df_enem_analise.printSchema()

root
 |-- NU_INSCRICAO: long (nullable = true)
 |-- NU_ANO: integer (nullable = true)
 |-- NO_MUNICIPIO_RESIDENCIA: string (nullable = true)
 |-- CO_UF_RESIDENCIA: integer (nullable = true)
 |-- SG_UF_RESIDENCIA: string (nullable = true)
 |-- NU_IDADE: integer (nullable = true)
 |-- TP_SEXO: string (nullable = true)
 |-- TP_ESTADO_CIVIL: integer (nullable = true)
 |-- TP_COR_RACA: integer (nullable = true)
 |-- TP_NACIONALIDADE: integer (nullable = true)
 |-- NO_MUNICIPIO_NASCIMENTO: string (nullable = true)
 |-- CO_UF_NASCIMENTO: integer (nullable = true)
 |-- SG_UF_NASCIMENTO: string (nullable = true)
 |-- TP_ST_CONCLUSAO: integer (nullable = true)
 |-- TP_ESCOLA: integer (nullable = true)
 |-- TP_ENSINO: integer (nullable = true)
 |-- IN_TREINEIRO: integer (nullable = true)
 |-- CO_UF_ESC: integer (nullable = true)
 |-- SG_UF_ESC: string (nullable = true)
 |-- TP_DEPENDENCIA_ADM_ESC: integer (nullable = true)
 |-- TP_LOCALIZACAO_ESC: integer (nullable = true)
 |-- IN_BAIXA_VISAO: in